In [ ]:
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz

In [ ]:
!python3 -m nltk.downloader stopwords
!python3 -m nltk.downloader universal_tagset
!python3 -m spacy download en # download the english model

In [ ]:
!pip install git+https://github.com/boudinfl/pke.git

In [ ]:
!pip install flair

In [ ]:
pip install git+https://github.com/LIAAD/yake

In [38]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Cell_Phones_and_Accessories_5.json.gz')

In [92]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4.0,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5.0,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5.0,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4.0,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5.0,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [93]:
df['asin'].unique()[99]

'B0009T8310'

In [94]:
myString = ". ".join(df[df['asin'] == "B0009T8310"]['reviewText'])

In [95]:
myString

"Im very pleased with this little charger. The blue light indicates whether or not it's on, and my phone charges very fast. Five minutes and it's three quarters charged from bring dead! Would definitely recommend this to someone else.. needed a power source with a mini usb. This is exactly what I needed. It was also delivered very fast.  I use this to power a RCA to HDMI converter for a Roku 3.. Yea, basically. It's a phone charger for the Motorola Razor, and guess what? It works for it. Also works for a few other phones.. The charger did come in and does charge my mp3 players like it's supposed to. The shipping was longer than what I paid for. However I thought the charger was a disconnected charger to where the cable could come off the base and connect to a computer so I'll have to order another charger that does what I need it to do.. It charged my phone.  Actually, I bought six of these so it would be more correct to say that &#34;they&#34; charged my phone.Why would I get six?  On

In [96]:
import pke

# define the valid Part-of-Speeches to occur in the graph
pos = {'NOUN', 'PROPN', 'ADJ'}

# define the grammar for selecting the keyphrase candidates
grammar = "NP: {<ADJ>*<NOUN|PROPN>+}"

# 1. create a PositionRank extractor.
extractor = pke.unsupervised.PositionRank()

# 2. load the content of the document.
extractor.load_document(input=myString,
                        language='en',
                        normalization=None)

# 3. select the noun phrases up to 3 words as keyphrase candidates.
extractor.candidate_selection(grammar=grammar,
                              maximum_word_number=3)

# 4. weight the candidates using the sum of their word's scores that are
#    computed using random walk biaised with the position of the words
#    in the document. In the graph, nodes are words (nouns and
#    adjectives only) that are connected if they occur in a window of
#    10 words.
extractor.candidate_weighting(window=10,
                              pos=pos)

# 5. get the 10-highest scored candidates as keyphrases
keyphrases = extractor.get_n_best(n=20)

In [102]:
keyphrases

[('little charger', 0.17963258659813622),
 ('phone charger', 0.16491456984372316),
 ('car charger', 0.15004268234792303),
 ('disconnected charger', 0.13687296123644305),
 ('charger', 0.1234681352040811),
 ('blue light', 0.10199684671092252),
 ('old razor phone', 0.06984128544535678),
 ('phone charges', 0.052826227762620595),
 ('phone', 0.041446434639642045),
 ('few other phones', 0.03770000962800429),
 ('motorola razor', 0.03427041603649379),
 ('car outlet', 0.03332629268987969),
 ('kindle touch', 0.030705002928885942),
 ('power source', 0.028569618608326568),
 ('mini usb', 0.028315003305972915),
 ('car', 0.02657454714384193),
 ('white touch screen', 0.02483761541993195),
 ('mp3 players', 0.023370517263208438),
 ('motorola razr', 0.021324748343225272),
 ('hdmi converter', 0.019626787633459604)]

In [106]:
kw_list = []
for kw in keyphrases:
    kw_list.append(kw)

In [ ]:
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings
from flair.data import Sentence

# create a StackedEmbedding object that combines glove and forward/backward flair embeddings
stacked_embeddings = StackedEmbeddings([
                                        WordEmbeddings('glove'),
                                        FlairEmbeddings('news-forward'),
                                        FlairEmbeddings('news-backward'),
                                       ])

In [107]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [108]:
kw_list = sorted(kw_list, key=lambda x: x[1])

In [109]:
tag_sent_dict = {}
sent_list = nltk.sent_tokenize(myString)

In [110]:
for tag_score in kw_list:
  tag = tag_score[0]
  tag_sent_dict[tag] = []
  for sent in sent_list:
    if tag in sent:
      tag_sent_dict[tag].append(sent)

In [111]:
def find_mean_vector(vect_list):
  vect = torch.zeros(4196)
  c = 1
  for i in vect_list:
        try:
            vect += i
            c += 1
        except KeyError:
            pass
  return (vect/c).numpy()

In [112]:
import torch
def get_mean_emb(phrase, we):
    ws = phrase.split(" ")
    vect = torch.zeros(4196)
    c = 1
    for i in ws:
        try:
            vect += we[i]
            c += 1
        except KeyError:
            pass
    return (vect).numpy()

In [113]:
import torch

In [114]:
tag_we_dict = {}

In [115]:
for tag, sentences in tag_sent_dict.items():
  vect_list = []
  for s in sentences:
    sentence = Sentence(s)
    stacked_embeddings.embed(sentence)
    we = {}
    for token in sentence:
      we[token.text] = token.embedding
    vect_list.append(get_mean_emb(tag, we))
  tag_we_dict[tag] = find_mean_vector(vect_list)

In [116]:
tag_we_dict

{'blue light': array([ 0.04038555,  0.63877   , -0.0832125 , ..., -0.00711576,
        -0.00816192, -0.00263372], dtype=float32),
 'car': array([-0.1263    , -0.4037025 ,  0.23366249, ...,  0.0041707 ,
        -0.03867931, -0.00891347], dtype=float32),
 'car charger': array([-0.53000504, -0.43840998,  0.36054   , ...,  0.00154909,
        -0.04838721, -0.01475241], dtype=float32),
 'car outlet': array([ 0.175955  , -0.510265  ,  0.1903235 , ...,  0.00545255,
        -0.0397969 , -0.00284723], dtype=float32),
 'charger': array([-7.9254228e-01, -3.0093336e-01,  3.6402667e-01, ...,
        -1.8343343e-04, -5.9669878e-02, -1.0834954e-02], dtype=float32),
 'disconnected charger': array([-0.48312402, -0.25509   ,  0.55013   , ..., -0.00089348,
        -0.02576532, -0.00536013], dtype=float32),
 'few other phones': array([-0.287005  ,  0.503855  ,  0.062452  , ..., -0.00521679,
        -0.04545446,  0.00114028], dtype=float32),
 'hdmi converter': array([0., 0., 0., ..., 0., 0., 0.], dtype=flo

In [117]:
from sklearn.cluster import KMeans
import numpy as np
from sklearn.cluster import DBSCAN

In [118]:
def normalize(v):
    norm=np.linalg.norm(v, ord=1)
    if norm==0:
        norm=np.finfo(v.dtype).eps
    return v/norm

In [119]:
embeddings = []
for vec in list(tag_we_dict.values()):
  embeddings.append(normalize(vec))

In [120]:
X = np.array(embeddings)

In [121]:
dbscan = DBSCAN(eps=0.5, min_samples=5, metric='euclidean', metric_params=None, algorithm='brute', leaf_size=30, p=None, n_jobs=None).fit(X)
dbscan.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [122]:
 kmeans = KMeans(n_clusters=8, random_state=0).fit(X)
kmeans.labels_

array([1, 1, 5, 4, 0, 6, 4, 1, 0, 1, 7, 2, 2, 2, 4, 3, 3, 0, 2, 3],
      dtype=int32)

In [123]:
p = np.array(list(tag_we_dict.keys()))

In [124]:
for i in range(8):
    print(p[kmeans.labels_ == i])
    print("  ")

['car' 'car outlet' 'car charger']
  
['hdmi converter' 'motorola razr' 'kindle touch' 'motorola razor']
  
['phone' 'phone charges' 'old razor phone' 'phone charger']
  
['charger' 'disconnected charger' 'little charger']
  
['white touch screen' 'power source' 'blue light']
  
['mp3 players']
  
['mini usb']
  
['few other phones']
  
